# Open WebUI 作为前端聊天框

上一节我用 3 种方式部署了本地大模型：Ollama, vLLM 和 transformers.

有了后端的推理服务，下一步该连接前端聊天框了。开源聊天框有很多，比如 AnythingLLM, LM Studio. 看了一圈，最推荐的还是 [Open WebUI](https://github.com/open-webui/open-webui). Open WebUI 有预开发的账号系统和历史对话模块，支持开发 RAG 和联网搜索。让我自己整出这么一个界面干净、布局合理的聊天框是不容易的。

Open WebUI 的界面是这样的：

![](./img/open_webui.jpg)


## 1. 安装 Open WebUI

Open WebUI 基本做到了开箱即用，但是部署它需要一些 Docker 基础。如果你对 [Docker](https://docs.docker.com/guides/) 不熟悉，可以参考我的两篇博客：

- [Docker 命令速查手册](https://luochang212.github.io/posts/docker_command/)：介绍 Docker 常用命令
- [Docker 部署 Spark 环境](https://luochang212.github.io/posts/synchrotrap/#%E9%99%84%E5%BD%951%E9%83%A8%E7%BD%B2-spark-%E7%8E%AF%E5%A2%83)：通过具体任务熟悉基础 Docker 操作 (GitHub: [SynchroTrap](https://github.com/luochang212/SynchroTrap/blob/main/%E9%99%84%E5%BD%951.%E9%83%A8%E7%BD%B2Spark%E7%8E%AF%E5%A2%83.ipynb))

**1）检查 Docker 环境**

确保已经安装 docker，且服务端已开启。如果使用 Windows，应该首先打开 Docker 桌面软件。

打开命令行输入：

```bash
docker info
```

如果 Docker 是正常运行的，你应该看到一长串 INFO 信息，既包含 Server 信息，也包含 Client 信息。

**2）下载镜像**

这步最困难的地方在于，它需要一点网络魔法。目前国内似乎没有一个云服务商，免费提供 Docker 镜像的镜像站服务。

```bash
docker pull ghcr.io/open-webui/open-webui:cuda
```

**3）创建容器**

下述命令使用镜像 `ghcr.io/open-webui/open-webui:cuda`，创建了一个容器。

```bash
docker run -d \
    --gpus all \
    -p 3215:8080 \
    -v ./data:/app/backend/data \
    --name open-webui \
    ghcr.io/open-webui/open-webui:cuda
```

参数解释：

- `-d`：表示在后台运行容器
- `--gpus all`：指定容器使用所有可用的 GPU 资源，一般用于多卡环境启用 GPU 支持
- `-p 3215:8080`：将本机的 3215 端口映射到容器的 8080 端口，实现外部访问容器内服务
- `-v ./data:/app/backend/data`：将本地 `./data` 路径挂载到容器的 `/app/backend/data` 路径，实现数据持久化
- `--name`：指定容器名称为 `open-webui`

⚠ 请注意：

1. 由于我们将本地 `./data` 路径挂载到容器的 `/app/backend/data` 路径，请确保 `./data` 目录已被创建
2. 由于我们将本机的 3215 端口映射到容器的 8080 端口，请确保本地的 3215 端口未被占用，如被占用需要更换端口

**4）打开前端页面**

浏览器打开 http://127.0.0.1:3215

可能需要稍微加载一会儿，UI 界面才会出现。

## 2. 为 WebUI 配置推理后端

**1）Ollama**

如果之前你已经运行 Ollama 模型，Ollama 会在默认端口 11434 提供服务。由于 Open WebUI 原生支持 Ollama，它会自动发现 11434 端口的 Ollama 服务。不需要做额外操作，就可以直接使用 Ollama 后端服务。

如果还没有运行 Ollama，执行以下命令就可以：

```bash
ollama run deepseek-r1:1.5b
```


**2）vLLM**

Open WebUI 支持 OpenAI API 格式，也就是说只要你的 API 符合 OpenAI 的格式，就很容易接入 Open WebUI
. 至于如何让自己的 API 符合 OpenAI 规范，可以参考这两个链接：

- OpenAI 文档：[Create chat completion](https://platform.openai.com/docs/api-reference/chat/create)
- OpenAI GitHub 仓库：[openai/openai-quickstart-python](https://github.com/openai/openai-quickstart-python)

vLLM 支持从命令行直接启动 OpenAI 风格的 API. 如果你和我一样，不想花时间写 API，可以用以下代码快速启动。`--model` 后面跟本地模型的路径，也许你需要改一下。

```bash
# 1. 激活 conda 环境
conda init && conda activate vllm_env

# 2. 启动 vLLM OpenAI API 服务
CUDA_VISIBLE_DEVICES=0 python3 -m vllm.entrypoints.openai.api_server \
    --model ./model/DeepSeek-R1-Distill-Qwen-1.5B \
    --served-model-name DeepSeek-R1-Distill-Qwen-1.5B \
    --gpu-memory-utilization 0.95 \
    --tensor-parallel-size 1 \
    --max-num-batched-tokens 51200 \
    --dtype auto \
    --enable-prefix-caching \
    --max-num-seqs 512 \
    --max-model-len 2048 \
    --host 0.0.0.0 \
    --port 9494 \
    --api-key token-abc123456

```

vLLM 启动通常需要花一点时间，我们可以在这时候顺便配一下 Open WebUI 的设置。

点左下角头像 -> Setting -> Admin Settings，然后进入 Connections 页面。把 OpenAI API 选项打开。再点击设置按钮进入 API 配置页。

> 补充：这里还要把 Ollama API 临时关掉，否则优先使用 Ollama 服务。

![](./img/open_webui_settings.jpg)

配置页一共要填三行：

- URL: http://host.docker.internal:9494/v1
- Key: token-abc123456
- Model IDs: DeepSeek-R1-Distill-Qwen-1.5B

填完 Model ID 还要按 + 号把记录添加上去，再按 Save 保存，配置才会生效。

![](./img/open_webui_add_model.jpg)

填完以后，就可以在模型列表里看到 `DeepSeek-R1-Distill-Qwen-1.5B` 了。

![](./img/open_webui_finish.jpg)

用 bash 代码直接启动的 vLLM 服务并不靠谱，经常出现乱码、截断等等问题。还是第一节中的 vLLM 配置和启动方法靠谱。所以下一节，我们用 FastAPI + vLLM 开发一个符合 OpenAI 规范的模型后端 API，这样就能组合出一个可用的 vLLM 服务。

可见当下 vLLM 生态与 DeepSeek 的兼容并不完美。如果只是想玩玩本地部署大模型 Ollama 足矣，折腾 vLLM 属实没必要。